# Feedback

First, you need to log the feedback from your app. An easy way to do this is to keep track of a run ID for each run, and then use that to log feedback. 


Keeping track of the run ID would look something like:

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="langsmith-basics"

In [17]:
from langchain_openai import ChatOpenAI
from langsmith import traceable


llm = ChatOpenAI(model="gpt-4o-mini")

# This is the retriever we will use in RAG
# This is mocked out, but it could be anything we want

@traceable(run_type="retriever")
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
@traceable(metadata={"llm": "gpt-3.5-turbo"})
def rag(question, **kwargs):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:
    
    {docs}""".format(docs="\n".join(docs))
    
    return llm(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )

In [18]:
rag("Where did Harrison work?")

AIMessage(content='Harrison worked at Kensho.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-63c53d45-944b-4431-9013-a991d225ea8f-0', usage_metadata={'input_tokens': 34, 'output_tokens': 7, 'total_tokens': 41})

In [19]:
import uuid

run_id = str(uuid.uuid4())
rag(
    "where did harrison work",
    langsmith_extra={"run_id": run_id}
)

AIMessage(content='Harrison worked at Kensho.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5c1ac46-7cd9-409b-9552-780f3b0ae469-0', usage_metadata={'input_tokens': 34, 'output_tokens': 7, 'total_tokens': 41})

In [20]:
from langsmith import Client
ls_client = Client()

ls_client.create_feedback(
    run_id,
    key="user-score",
    score=1.0,
)

Feedback(id=UUID('8735ecf4-08d1-4048-844d-1012c6d5c2c8'), created_at=datetime.datetime(2024, 8, 17, 3, 18, 5, 155354, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 8, 17, 3, 18, 5, 155354, tzinfo=datetime.timezone.utc), run_id=UUID('1127b6e6-a382-4623-91fa-ef36705a0f29'), key='user-score', score=1.0, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None)